# Warming projections from FRIDA

Run the forcing from FRIDA through the calibrated FaIR model and compare FRIDA's warming against this.

In [ ]:
import os
import fair
from fair.interface import fill, initialise
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np

In [ ]:
pl.rcParams['figure.figsize'] = (8.7/2.54, 8.7/2.54)
pl.rcParams['font.size'] = 7
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['ytick.direction'] = 'in'
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.direction'] = 'in'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['xtick.major.top'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['axes.spines.top'] = True
pl.rcParams['axes.spines.bottom'] = True
pl.rcParams['figure.dpi'] = 150

In [ ]:
fair.__version__

In [ ]:
fair.__file__

In [ ]:
df_frida = pd.read_csv('../data/frida0.1-20230313-warming-forcing.csv', index_col=0)

In [ ]:
df_frida.index.name = 'year'

In [ ]:
df_frida

In [ ]:
pl.plot(df_frida['Observed warming'])

In [ ]:
pl.plot(df_frida['GHG forcing'])  # incorrectly named

In [ ]:
df_configs = pd.read_csv('../data/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
df_configs.index.name = 'config'

In [ ]:
df_configs

In [ ]:
f = fair.FAIR()

In [ ]:
f.define_time(1980, 2100, 1/8)

In [ ]:
f.timebounds

In [ ]:
f.define_scenarios(['frida'])

In [ ]:
df_configs.index

In [ ]:
f.define_configs(df_configs.index)

In [ ]:
# Create an aggregate forcing time series specie.
f.define_species(['Fext'],
    {
        'Fext': {
            'type': 'unspecified',
            'input_mode': 'forcing',
            'greenhouse_gas': False,
            'aerosol_chemistry_from_emissions': False,
            'aerosol_chemistry_from_concentration': False
        }
    }
)

In [ ]:
f.allocate()

In [ ]:
f.forcing.coords

In [ ]:
f.forcing.shape

In [ ]:
f.forcing[:, 0, :, 0] = df_frida['GHG forcing'].values[:, None]

In [ ]:
# import configs into fair
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(f.climate_configs["ocean_heat_transfer"], df_configs.loc[:, "kappa1":"kappa3"].values)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

Note: we have initialised FaIR to be 0.4C in all ocean layers in 1980; effectively assuming the 1980 conditions are equilibrium. This isn't correct as in 1980 we are in a transient regime; the ocean has warmed by a non-zero, but less than equilibrium, amount.

In [ ]:
df_fair1980 = pd.read_csv('../output/fair_1980_state.csv', index_col=0)
df_fair1980

In [ ]:
# initial conditions
initialise(f.forcing, df_frida.loc[1980, 'GHG forcing'])
fill(f.temperature, df_fair1980.values, timebounds=1980, scenario='frida')
#initialise(f.temperature, 0)
#fill(f.temperature, df_frida.loc[1980, 'Observed warming'], layer=0, timebounds=1980)

In [ ]:
f.run()

In [ ]:
df_ar6 = pd.read_csv('../data/AR6_GMST.csv', index_col=0)
df_ar6

In [ ]:
pl.fill_between(
    f.timebounds, 
    np.percentile(f.temperature[:, 0, :, 0], 5, axis=1), 
    np.percentile(f.temperature[:, 0, :, 0], 95, axis=1),
    color='0.8',
    label = 'FAIR 5-95%'
)
pl.fill_between(
    f.timebounds, 
    np.percentile(f.temperature[:, 0, :, 0], 17, axis=1), 
    np.percentile(f.temperature[:, 0, :, 0], 83, axis=1),
    color='0.6',
    label = 'FAIR 17-83%'
)
pl.plot(f.timebounds, np.median(f.temperature[:, 0, :, 0], axis=1), color='k', label='FAIR median')
pl.plot(df_frida['Observed warming'], color='r', label='FRIDA')
pl.plot(df_ar6.index[130:]+0.5, df_ar6.loc[1980:, 'gmst'], color='b', label='Observations')
pl.legend(loc="upper left")
pl.xlim(1980, 2100)
pl.ylim(0, 3.5)
pl.ylabel('°C relative to 1850-1900')
pl.title('Global mean surface temperature anomaly')
pl.tight_layout()

os.makedirs('../plots/', exist_ok=True)
pl.savefig('../plots/frida_fair_climate.png')